# OpenSWPC トライアル@Google Colab

地震波動伝播数値シミュレーションコード [OpenSWPC](https://github.com/OpenSWPC/OpenSWPC) のダウンロードとコンパイルから簡単な実行までを，Google Colab上で試してみましょう．

## Google Colab とその利用について

Google Colaboratory (Google Colab) は，Googleが提供する機械学習のためのプラットフォームで，ブラウザ上でPythonコードを実行できます．OpenSWPCそのものはPythonで書かれているわけではないのですが，実はGoogle ColabはそのバックエンドにLinuxの仮想マシンが動いており，すこしの工夫でクラウド上のLinux環境としても利用ができるのです．ここでは，ColabのLinuxとしての機能を使い，必要なライブラリをインストールしてからOpenSWPCをコンパイルし，実行してみます．

### 最低限のColabの操作とLinux環境の確認

Google Colabでは **セル** と呼ばれる単位で文章やコードを管理しています．今読んでいるこの文字列の部分を1度クリックすると，文章全体の周辺に灰色の枠線が表示されるはずです．その囲まれた全体がひとつのセルです．

今度は，文書をダブルクリックしてみると，左側にMarkdown形式の原稿が，右側にそのプレビューが表示される編集画面になります．Markdown形式は，テキストに簡易な記号による修飾ができるようにしたものです（ここではその詳細は紹介しません）．他のセル部分をクリックするか，あるいは <kbd>Shift</kbd> キーと <kbd>Enter</kbd>キーを同時に押すと元の表示状態に戻ります．

In [ ]:
# 左側に実行ボタン (▷) が表示されているこの部分は，コードセルです．
# ここにPythonの文法でなにかの命令を書いて，実行ボタンを押すか，
# あるいはShift+Enterキーを押すとその命令が実行されます．
# 実際に実行ボタンを押してみましょう．下記の計算が実行され，その下に結果が表示されるはずです．

3 + 5

Linuxのコマンドを実行するには，コードセルの行の先頭に `!` を書くか，あるいはセルの先頭行に `%%bash` と書きます．前者は行単位で，後者はセル単位で，そこに書かれている命令が `bash` の命令として実行されます．


In [ ]:
! echo "This is a Linux command."

In [ ]:
%%bash

echo "This is another Linux command"

これらのLinuxコマンドがどのような環境で実行されているのか調べてみましょう． `uname` コマンドは現在使用しているLinuxのカーネルバージョンを表示するものです．

In [ ]:
! uname -a

この結果表示から，このColab環境が `x86_64` （通常のWindowsやLinux PCで使われているのと同じハードウェア環境）でのLinuxであることがわかります．

さらに踏み込んで，OSの種類を表示してみます．

In [ ]:
! cat /etc/os-release

Linuxのなかでも非常によく使われている，Ubuntuのバージョン22.04 LTS（2022年4月にリリースされた長期サポートバージョン）であることがわかりました．UbuntuはWindows上でLinuxを動作させる Windows Subsystem for Linux (WSL) でも標準として採用されているものです．すなわち，以下に紹介するOpenSWPCのコンパイル方法は，WSL上のUbuntuでもほとんど同じように実行できます．

普通はUbuntu Linuxの初期状態ではFortranは入っていない（ことが多い；セットアップ時の設定次第）のですが，ここではもう `gfortran` が使えます．

In [ ]:
%%bash

# echo コマンドでFortranプログラムソースコードを記述し
# リダイレクト (>) を用いて test.f90 ファイルに保存する

echo "program test

    write(*,*) 'Hello gfortran!'

end program test
" > test.f90

In [ ]:
! cat test.f90

In [ ]:
# コンパイルして実行バイナリ a.out を作成
! gfortran test.f90

In [ ]:
# コンパイルされた実行バイナリを実行
! ./a.out

## OpenSWPCのセットアップ



### 準備：NetCDFのインストール

OpenSWPCの利用にはNetCDFのライブラリが必要で，さすがにそれはGoogle Colabには入っていません．そこで，Ubuntuのパッケージ管理システムコマンド `apt` を使って，インストールしてしまいましょう．いくらかシステムメッセージが表示されますが，ここではスルーして大丈夫です．

In [ ]:
# NetCDFの開発用ライブラリとそのFortran用をインストール
# オプションの -qq は画面表示のメッセージ量を少なくするためのもの
! apt -qq install libnetcdf-dev libnetcdff-dev

つづけて，OpenSWPC公開ファイルをダウンロードしてきて，展開します．GUIでダウンロードしたファイルをColabにドラッグドロップでアップロードしてもよいのですが，ここではすべてコマンドでやってみます．

In [ ]:
%%bash

# 公開コードのダウンロード
wget https://github.com/OpenSWPC/OpenSWPC/archive/refs/tags/25.01.zip -O 25.01.zip -q

# ダウンロードされた圧縮データの展開
unzip -oq 25.01.zip

これで，`OpenSWPC-25.01` というディレクトリが作成され，その下にソースコード一式が含まれた状態になっています．ブラウザ画面左側のフォルダアイコンをクリックするとファイルとディレクトリの一覧が確認できます．

それでは，コンパイルです．OpenSWPCは`make` コマンドでコンパイルしますが，関連ライブラリのインストール場所のOSによる違いを`make` のオプションパラメタ`arch`として与えます．Ubuntuの場合，プリセットが用意されていますので，以下のコマンドを実行するだけです．コンパイルには1〜2分かかります．コンパイルが終わると，`OpenSWPC-25.01/bin` 以下に '.x' 拡張子の実行ファイルがたくさん出来上がります．これらが計算コードならびに周辺ツール群です．

In [ ]:
%%bash

# まずはソースコード格納ディレクトリに cd コマンドで移動して
cd OpenSWPC-25.01/src

# makeコマンドですべてのコードをコンパイル
make arch=ubuntu-gfortran

## OpenSWPCを動かしてみる

`OpenSWPC-25.01/example` には，いくつかお試し用の入力パラメタが含まれています．まずはそれをそのまま動かしてみましょう．ここでは，鉛直（深さ）断面におけるP-SV問題を解く `swpc_psv.x` を実行してみます．

In [ ]:
%%bash

cd OpenSWPC-25.01

mpirun --allow-run-as-root --oversubscribe -np 2 ./bin/swpc_psv.x -i example/input.inf

50秒ほどで実行が終わるでしょう．出力として表示されるのは，計算中の一定時間ステップごとの計算時間，予想残り計算時間，そして空間中の最大振幅です．本来は計算の進行とともに逐次的に表示されるべきものですが，Colab環境では計算後にまとめての表示となってしまうようです．

計算を実行する際の `mpirun` は，複数のCPUあるいはCPUコアを使って **並列計算** するときに必要なものです．その後のオプション `-np` で並列に動作させる計算（プロセス）の数を指定しています．

その他のオプション（`--allow-run-as-root`,  `--oversubscribe`）はいずれもColab上で計算を行うために特に必要なものです．同じ計算をローカルのLinuxマシンやWSL2上で実行する場合には，これらのオプションは必要ないはずです．

実行が終わると，`OpenSWPC-25.01/out` ディレクトリ以下に結果ファイルが出力されます． `.nc` 拡張子のファイルが空間スナップショットです．他に波形データが `sac` 形式で `out/wav` に保存されています．

ここでは，付属ツール `read_snp.x` を使ってスナップショットを可視化してみます．

In [ ]:
%%bash

cd OpenSWPC-25.01/out
../bin/read_snp.x -i ./swpc.psv.xz.v.nc -bmp -pall -mul 20 >& /dev/null

上記コマンドで `out/swpc/psv/xz/v2/` 以下にビットマップ形式の画像が生成されます．一つずつの画像をクリックして見ることもできますが，ここでは`ffmpeg` というツールを使って，まとめてgifアニメーションにしてみましょう．

In [ ]:
%%bash

cd OpenSWPC-25.01
ffmpeg -i ./out/swpc/psv/xz/v2/swpc.psv.xz.v2.%6d.bmp \
-qscale 0 -vf "scale=400:-1,split [a][b];[a] palettegen [p];[b][p] paletteuse" \
-pix_fmt yuv420p -y swpc.gif >& /dev/null

この可視化では，鉛直方向と水平方向の変位速度の絶対値をそれぞれ赤色と緑色で表現しています．

ここまでの一連の流れを簡単に実行できるよう，関数にまとめておきます．

In [ ]:
def exe_psv():
  import os
  import sys
  import IPython

  print('Running OpenSWPC ...', file=sys.stderr)
  cmd  = 'cd OpenSWPC-25.01; '
  cmd += 'mpirun --allow-run-as-root --oversubscribe -np 2 ./bin/swpc_psv.x -i example/input.inf'
  os.system(cmd)


  print('Visualizing ...', file=sys.stderr)
  cmd  = 'cd OpenSWPC-25.01/out; '
  cmd += '../bin/read_snp.x -i ./swpc.psv.xz.v.nc -bmp -pall -mul 20'

  os.system(cmd)

  print('Making GIF movie ...', file=sys.stderr)
  cmd  = 'cd OpenSWPC-25.01/out; '
  cmd += 'ffmpeg -i ./swpc/psv/xz/v2/swpc.psv.xz.v2.%6d.bmp '
  cmd += '-qscale 0 -vf "scale=400:-1,split [a][b];[a] palettegen [p];[b][p] paletteuse" '
  cmd += '-y swpc.gif'

  os.system(cmd)

  return IPython.display.Image('OpenSWPC-25.01/out/swpc.gif')

ためしに実行してみます．先ほど確認したのと同じ動画が表示されるはずです．

In [ ]:
exe_psv()

## パラメタファイルとその調整

これまでに動かしたプログラムの動作は，基本的にはパラメタファイル `example/input.inf` で定義されています．このなかで，並列計算の設定やグリッド数，時間ステップ数などの基本パラメタのほか，速度構造モデルや震源モデルのチョイス等々を設定しています．

さらに，`input.inf` 内でファイル名を指定して呼び出される設定ファイルとして，

- `lhm.dat` 成層構造を指定するファイル
- 'source.dat' 震源の位置とメカニズムを指定するファイル
- 'stloc.xy` 観測点位置の指定

があります．

ここでは練習として，以下の変更を行なってみましょう．

1. `input.inf` からパラメタ `nx` を2倍に，`nz` を半分にする．
> これによって水平方向に2倍の距離の計算が実施できます．待てるのでしたら時間ステップ `nt` も倍にするとよいでしょう．

2. `input.inf` から速度構造ファイル `lhm.dat` を呼び出しているところを探し，それを別のファイル `lhm_jma.dat` に変更してみる．
> 前者は防災科研F-netのCMT解析に使われている速度構造，後者は気象庁JMA2001モデルのP波・S波速度構造に経験的関係により質量密度とQ値を設定したもの

3. `source.dat` を開き，コメント `#` のついていない行を探し，`z=10 km`, `mxx=myy=mzz=1.0`, `mxz=1.0` にしてみる
> 震源の深さを深くするとともに，モーメントテンソルの大きさ（地震モーメントあるいはモーメントマグニチュード）の大きさを変えないままメカニズム解を変更しました．これは断面内の傾斜角45度の逆断層に相当します．



In [ ]:
exe_psv()

深さとともに滑らかに変化する速度構造になったことと，S波が震源からより強く放射される（元の例は膨張震源でP波のみが放射され，それが地表面で変換してS波になっていました）こと，また時間ステップを長くすると分散性の表面波が地表付近だけを揺らしながら水平に伝わっていくこともよく見えるでしょう．

OpenSWPCではこんなふうにパラメタを設定することで簡単にモデルや震源を差し替えながら地震動モデリングができます．この例では地震波形記録は扱いませんでしたが，波形は[SAC (Seismic Analysis Code)](https://ds.iris.edu/ds/nodes/dmc/software/downloads/sac/)という理学の地震学研究で広く使われている波形処理プログラムの形式で出力されます．SACをお使いであればそのまま利用できますし，Python上での地震波形解析ライブラリのデファクトスタンダードである [ObsPy](https://docs.obspy.org/)から読むことも出来ます．Fortranをお使いの方には，OpenSWPC開発者による [sacpack](https://github.com/tktmyd/sacpack) というサブルーチン集もあります．